In [15]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama

In [2]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

MODEL_LLAMA = "llama3.2"

OpenAI API Key exists and begins sk-proj-


In [3]:
system_prompt = "You are an helpful assistant known as FlightAI"
system_prompt += "reply no more than 1 senetnce"
system_prompt += "Be accurate, if don't know the answer just say so"

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [5]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [6]:
tools = [{"type": "function", "function": price_function}]

In [25]:
def chat(message, history, model='llama'):
    if model == "openai":
        messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            response, city = handle_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
        return response.choices[0].message.content
    elif model == "llama":
        messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
        response = ollama.chat(model=MODEL_LLAMA, messages=messages, tools=tools)
        if 'message' in response and isinstance(response['message'], dict) and 'content' in response['message']:
            content = response['message']['content']
            if isinstance(content, dict) and 'finish_reason' in content:
                if content['finish_reason'] == "tool_calls":
                    message = response['message']['content']
                    response, city = handle_tool_call(message)
                    messages.append(message)
                    messages.append(response)
                    response = ollama.chat(model=MODEL_LLAMA, messages=messages)
                return  response['message']['content']

In [26]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [27]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Aquib\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aquib\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aquib\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aquib\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1633, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aquib\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py", line 850, in async_wrapper
    response = await f(*args, **kwargs)
              